### detect

In [1]:
import time
import shutil
import random
from pathlib import Path
from models import *
from utils.datasets import *
from utils.utils import *
from utils import torch_utils
from torch.utils.data import DataLoader

In [2]:
cfg = 'cfg/yolov3-spp.cfg'
data_cfg = 'data/coco.data'
weights = 'weights/yolov3-spp.weights'
images = 'samples/'
output = 'output'
save_images=True
img_size = 416
conf_thres = 0.5
nms_thres = 0.5

In [13]:
# https://github.com/ultralytics/yolov3/blob/master/detect.py
def detect(
        cfg=cfg,
        data_cfg=data_cfg,
        weights=weights,
        images=images,
        output=output,
        img_size=img_size,
        conf_thres=conf_thres,
        nms_thres=nms_thres,
        save_images=True
        ):
    
    device = torch_utils.select_device()
    
    if os.path.exists(output):
        shutil.rmtree(output)  # delete output folder
    os.makedirs(output)  # make new output folder
    
    # Initialize model
    model = Darknet(cfg, img_size)
    
    # Load weights
    if weights.endswith('.pt'):  # pytorch format
        model.load_state_dict(torch.load(weights, map_location=device)['model'])
    else:  # darknet format
        _ = load_darknet_weights(model, weights)
        
    model.to(device).eval()
    
    # Set Dataloader
    vid_path, vid_writer = None, None
    
    dataloader = LoadImages(images, img_size=img_size)
    
    # Get classes and colors
    classes = load_classes(parse_data_cfg(data_cfg)['names'])
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(classes))]
    
    for i, (path, img, im0, vid_cap) in enumerate(dataloader):
        t = time.time()
        #print(path)
        save_path = str(Path(output) / Path(path).name)

        # Get detections
        img = torch.from_numpy(img).unsqueeze(0).to(device) # convert it to tensor and add 0 dim(batch)
        #print(img.shape)

        pred, _ = model(img)
        detections = non_max_suppression(pred, conf_thres, nms_thres)[0]

        if detections is not None and len(detections) > 0:
            # Rescale boxes from 416 to true image size
            scale_coords(img_size, detections[:, :4], im0.shape).round()

            # Print results to screen
            for c in detections[:, -1].unique():
                n = (detections[:, -1] == c).sum()
                print('%g %ss' % (n, classes[int(c)]), end=', ')

            # Draw bounding boxes and labels of detections
            for *xyxy, conf, cls_conf, cls in detections:
                # Add bbox to the image
                label = '%s %.2f' % (classes[int(cls)], conf)
                plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])

        print('Done. (%.3fs)' % (time.time() - t))

        if save_images:
            cv2.imwrite(save_path, im0)

In [14]:
detect()

Using CPU.
image 1/15 samples\Adrian.jpg: 2 persons, 8 cars, 3 potted plants, 1 traffic lights, Done. (1.083s)
image 2/15 samples\India.jpg: 8 persons, 1 cell phones, Done. (1.218s)
image 3/15 samples\Starks.jpg: 3 persons, Done. (1.217s)
image 4/15 samples\Trump.jpg: 2 persons, 1 ties, Done. (1.190s)
image 5/15 samples\Vector.jpg: 12 persons, Done. (1.500s)
image 6/15 samples\die-welle.jpg: 10 persons, Done. (1.226s)
image 7/15 samples\dog.jpg: 1 dogs, 1 bicycles, 1 trucks, Done. (1.286s)
image 8/15 samples\got-s8.jpg: 13 persons, Done. (1.338s)
image 9/15 samples\lifelines.jpg: 2 persons, 1 ties, Done. (1.251s)
image 10/15 samples\merlin.jpg: 1 trucks, 1 cars, 9 persons, 1 fire hydrants, 2 handbags, 1 dogs, Done. (1.269s)
image 11/15 samples\pornstar.jpg: 1 persons, Done. (1.208s)
image 12/15 samples\timg.jpg: 2 persons, Done. (1.315s)
image 13/15 samples\timg3.jpg: 4 persons, Done. (1.239s)
image 14/15 samples\walkers.jpg: 2 persons, Done. (1.170s)
image 15/15 samples\zidane.jpg: 2 

In [11]:
def My_detect(
        cfg=cfg,
        data_cfg=data_cfg,
        weights=weights,
        images=images,
        output=output,
        img_size=img_size,
        conf_thres=conf_thres,
        nms_thres=nms_thres,
        save_images=True
        ):
    
    device = torch_utils.select_device()
    
    if os.path.exists(output):
        shutil.rmtree(output)  # delete output folder
    os.makedirs(output)  # make new output folder
    
    # Initialize model
    model = Darknet(cfg, img_size)
    
    # Load weights
    if weights.endswith('.pt'):  # pytorch format
        model.load_state_dict(torch.load(weights, map_location=device)['model'])
    else:  # darknet format
        _ = load_darknet_weights(model, weights)
        
    model.to(device).eval()
    
    # Set Dataloader
    vid_path, vid_writer = None, None
    
    #dataloader = LoadImages(images, img_size=img_size)
    dataset = MyLoadImages(images, img_size=img_size) # Pytorch dataset
    dataloader = DataLoader(dataset, batch_size=1, num_workers=1, shuffle=False) # Pytorch dataloader
    
    # Get classes and colors
    classes = load_classes(parse_data_cfg(data_cfg)['names'])
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(classes))]
    
    for i, (path, img, im0) in enumerate(dataloader):
        t = time.time()
        path = ''.join(path)
        save_path = str(Path(output) / Path(path).name)

        # Get detections
        #img = torch.from_numpy(img).unsqueeze(0).to(device)
        img = img.to(device) # already tensor, shift to device
        #print(img.shape)
        im0 = im0.squeeze(0).numpy() # convert it to numpy and remove batch dim

        pred, _ = model(img)
        detections = non_max_suppression(pred, conf_thres, nms_thres)[0]

        if detections is not None and len(detections) > 0:
            # Rescale boxes from 416 to true image size
            scale_coords(img_size, detections[:, :4], im0.shape).round()

            # Print results to screen
            print('{}/{} image: {} '.format(i, len(dataset), path), end='')
            for c in detections[:, -1].unique():
                n = (detections[:, -1] == c).sum()
                print('%g %ss' % (n, classes[int(c)]), end=', ')

            # Draw bounding boxes and labels of detections
            for *xyxy, conf, cls_conf, cls in detections:
                # Add bbox to the image
                label = '%s %.2f' % (classes[int(cls)], conf)
                plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])

        print('Done. (%.3fs)' % (time.time() - t))

        if save_images:
            cv2.imwrite(save_path, im0)

In [12]:
My_detect()

Using CPU.
0/15 image: samples/Adrian.jpg 2 persons, 8 cars, 3 potted plants, 1 traffic lights, Done. (1.092s)
1/15 image: samples/die-welle.jpg 10 persons, Done. (1.178s)
2/15 image: samples/dog.jpg 1 dogs, 1 bicycles, 1 trucks, Done. (1.568s)
3/15 image: samples/got-s8.jpg 13 persons, Done. (1.175s)
4/15 image: samples/India.jpg 8 persons, 1 cell phones, Done. (1.148s)
5/15 image: samples/lifelines.jpg 2 persons, 1 ties, Done. (1.245s)
6/15 image: samples/merlin.jpg 1 trucks, 1 cars, 9 persons, 1 fire hydrants, 2 handbags, 1 dogs, Done. (1.365s)
7/15 image: samples/pornstar.jpg 1 persons, Done. (1.304s)
8/15 image: samples/Starks.jpg 3 persons, Done. (1.259s)
9/15 image: samples/timg.jpg 2 persons, Done. (1.240s)
10/15 image: samples/timg3.jpg 4 persons, Done. (1.156s)
11/15 image: samples/Trump.jpg 2 persons, 1 ties, Done. (1.194s)
12/15 image: samples/Vector.jpg 12 persons, Done. (1.291s)
13/15 image: samples/walkers.jpg 2 persons, Done. (1.259s)
14/15 image: samples/zidane.jpg 2 p